Source: https://www.kaggle.com/petrov/first-long-paragraph

Loading the required packages

In [ ]:
import json
from pathlib import Path
from tqdm import tqdm

import pandas as pd

TEST_TOTAL = 692

In [ ]:
PATH = Path('/kaggle/input/tensorflow2-question-answering')
!ls -1 {PATH}

The following function returns back the start & end tokens in the dataset

In [ ]:
def get_joined_tokens(answer: dict) -> str:
    return '%d:%d' % (answer['start_token'], answer['end_token'])

*get_pred*
This function will always return back - *YES* for a short_answer

As the name of the notebook suggests. The largest answer candidate is chosen OR if no answer candidate is found to be greater than 35 in length then the first answer candidate is picked. 

>If start of token as paragraph token 
    
    and *top_level* is True
    
    and len of answer candidate is >35 
    
    Then chose the answer 
If any of the above options do not work then select the first answer candidate
    

In [ ]:
candidates = json_data['long_answer_candidates']
candidates

In [ ]:
def get_pred(json_data: dict) -> dict:
    ret = {'short': 'YES', 'long': ''}
    candidates = json_data['long_answer_candidates']
    
    paragraphs = []
    tokens = json_data['document_text'].split(' ')
    for cand in candidates:
        start_token = tokens[cand['start_token']]
        if start_token == '<P>' and cand['top_level'] and cand['end_token']-cand['start_token']>35:
            break
    else:
        cand = candidates[0]
        
    ret['long'] = get_joined_tokens(cand)
    
    id_ = str(json_data['example_id'])
    ret = {id_+'_'+k: v for k, v in ret.items()} 
    return ret

In [ ]:
# Initialize a dictionary for holding all predictions

preds = dict()

In [ ]:
with open(PATH / 'simplified-nq-test.jsonl', 'r') as f:
    for line in tqdm(f, total=TEST_TOTAL):
        json_data = json.loads(line) 
        #break
        prediction = get_pred(json_data)
        preds.update(prediction)

In [ ]:
len(preds)

In [ ]:
submission = pd.read_csv(PATH / 'sample_submission.csv')
submission['PredictionString'] = submission['example_id'].map(lambda x: preds[x])
submission.to_csv('submission.csv', index=False)